In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from sklearn.preprocessing import StandardScaler

In [4]:
tf.__version__

'2.19.0'

In [5]:
tf.debugging.set_log_device_placement(False) # 打印变量分配

In [6]:
tf.config.experimental.list_logical_devices() # 逻辑设备

[LogicalDevice(name='/device:CPU:0', device_type='CPU')]

In [7]:
tf.config.experimental.list_physical_devices() # 物理设备

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

# gpu用多少申请多少

In [26]:
gups = tf.config.experimental.list_physical_devices('GPU')
for gup in gups:
  tf.config.experimental.set_memory_growth(gup,True)

# 设置用那个gpu

In [28]:
tf.config.experimental.set_visible_devices(gups[0], 'GPU')

IndexError: list index out of range

# 设计虚拟gpu

In [30]:
tf.config.experimental.set_virtual_device_configuration(
    gups[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)], # 虚拟gpu的显存
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)] # 虚拟gpu的显存
)

IndexError: list index out of range

In [10]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images_all, train_labels_all), (test_images, test_labels) = fashion_mnist.load_data()

In [11]:
valid_images = train_images_all[:5000]
valid_labels = train_labels_all[:5000]
train_images = train_images_all[5000:]
train_labels = train_labels_all[5000:]

In [12]:
print(train_images.shape,train_labels.shape)
print(valid_images.shape,valid_labels.shape)
print(test_images.shape,test_labels.shape)

(55000, 28, 28) (55000,)
(5000, 28, 28) (5000,)
(10000, 28, 28) (10000,)


In [13]:
train_images.astype(np.float32).reshape(train_images.shape[0], -1).shape

(55000, 784)

In [14]:
scaler = StandardScaler()

In [15]:
train_data_temp = train_images.astype(np.float32).reshape(train_images.shape[0], -1)
train_images_scaled = scaler.fit_transform(train_data_temp)
train_images_scaled.shape

(55000, 784)

In [16]:
valid_data_temp = valid_images.astype(np.float32).reshape(valid_images.shape[0], -1)
valid_images_scaled = scaler.fit_transform(valid_data_temp)
valid_images_scaled.shape

(5000, 784)

In [17]:
test_data_temp = test_images.astype(np.float32).reshape(test_images.shape[0], -1)
test_images_scaled = scaler.fit_transform(test_data_temp)
test_data_temp.shape

(10000, 784)

In [18]:
def make_dataset(data,target,epochs,batch_size,shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((data,target))
    if shuffle:
      dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [20]:
batch_size = 64
epochs = 10
train_dataset = make_dataset(train_images_scaled,train_labels,epochs,batch_size)
valid_dataset = make_dataset(valid_images_scaled,valid_labels,epochs,batch_size,shuffle=False)

In [21]:
(train_images_scaled.shape[1],)

(784,)

In [22]:
model = keras.models.Sequential()
with tf.device('/gpu:0'):
  model.add(keras.layers.Dense(512,activation='relu',input_dim=train_images_scaled.shape[1]))
  model.add(keras.layers.Dense(256,activation='relu'))
with tf.device('/gpu:1'):
  model.add(keras.layers.Dense(256,activation='relu'))
  model.add(keras.layers.Dense(256,activation='relu'))
with tf.device('/gpu:2'):
  model.add(keras.layers.Dense(256,activation='relu'))

model.add(keras.layers.Dense(10,activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
model.fit(
    train_dataset,
    epochs=10,
    validation_data=valid_dataset,
    steps_per_epoch=train_images_scaled.shape[0]//batch_size,
    validation_steps=valid_images_scaled.shape[0]//batch_size
    )

Epoch 1/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9465 - loss: 0.1442 - val_accuracy: 0.8938 - val_loss: 0.3692
Epoch 2/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9474 - loss: 0.1377 - val_accuracy: 0.8906 - val_loss: 0.3864
Epoch 3/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9528 - loss: 0.1222 - val_accuracy: 0.8964 - val_loss: 0.4041
Epoch 4/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9545 - loss: 0.1217 - val_accuracy: 0.8956 - val_loss: 0.3932
Epoch 5/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9592 - loss: 0.1053 - val_accuracy: 0.8988 - val_loss: 0.4420
Epoch 6/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9622 - loss: 0.1039 - val_accuracy: 0.8988 - val_loss: 0.4445
Epoch 7/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9632 - loss: 0.0968 - val_accuracy: 0.8934 - val_loss: 0.4721
Epoch 8/10
859/859 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9633 - loss: 0.0992 - val_acc